In [ ]:
"""

ANNOTATION SCIRPT/FORMAT/PROCEDURE FROM:

https://github.com/hollyjackson/casualty_mentions_nyt/tree/main

"""

In [29]:
import sys

import os
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import locale

locale.setlocale(locale.LC_ALL, 'nl_NL.UTF-8')
import datetime as dt

# NLTK imports
import nltk

nltk.data.path.append('../nltk_data/')
import string
from nltk import collocations
from nltk.text import Text
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk import RegexpParser
from nltk.tree import *

# spaCy imports
import spacy
from spacy.symbols import nsubj, VERB

from time import sleep

In [22]:
#PALESTINE_IDENTIFIERS = ["Palestine", "Palestinian", "Palestinians"]
PALESTINE_IDENTIFIERS = ["Palestina", "Palestijns", "Palestijnen", "Gazanen", "Westelijke Jordaanoever", "Gaza", "Hamas", "Gaza Strook", "Noord-Gaza", "Centraal-Gaza", "Zuid-Gaza"]
#ISRAEL_IDENTIFIERS = ["Israel", "Israeli", "Israelis"]
ISRAEL_IDENTIFIERS = ["Israël", "Israëlisch", "Israëliërs", "Noord-Israël", "Centraal-Israël", "Zuid-Israël"]

# Cities in the West Bank and Gaza
PALESTINIAN_CITIES = ["Jerusalem", "Abasan al-Kabira", "Abu Dis", "Bani Na\'im", "Bani Suheila",
                      "Beit Hanoun", "Beit Jala", "Beit Lahia", "Beit Sahour", "Beit Ummar", "Beitunia", "Bethlehem",
                      "Beit Lahm", "al-Bireh", "Deir al-Balah", "ad-Dhahiriya", "Dura", "Gaza Stad", "Ghazzah",
                      "Halhul",
                      "Hebron", "al-Khalil", "Idhna", "Jabalia", "Jenin", "Jericho", "Ariha", "Khan Yunis", "Nablus",
                      "Qabatiya", "Qalqilya", "Rafah", "Ramallah", "Sa\'ir", "as-Samu", "Surif", "Tubas", "Tulkarm",
                      "Ya\'bad", "al-Yamun", "Yatta", "az-Zawayda", "Nazareth", "Jaljulia", "Kafr Bara", "Kafr Qasim",
                      "Qalansawe", "Tayibe", "Tira", "Zemer", "Ar\'ara", "Baqa al-Gharbiyye", "al-Arian", "Basma",
                      "Jatt",
                      "Kafr Qara", "Ma\'ale Iron", "Meiser", "Umm al-Fahm", "Umm al-Qutuf", "Lod", "Ramla",
                      "Wadi Nisnas",
                      "Halisa", "Kababir", "Abbas", "Daliyat al-Karmel", "Ein Hawd", "Fureidis", "Ibtin", "Isfiya",
                      "Jisr az-Zarqa", "Khawaled", "Abu Ghosh", "Beit Jimal", "Ein Naqquba", "Ein Rafa", "Beit Hanina",
                      "Beit Safafa", "Jabel Mukaber", "Old City", "Ras al-Amud", "Sheikh Jarrah", "Shuafat", "Silwan",
                      "Sur Baher", "At-Tur", "Umm Tuba", "Wadi al-Joz", "al-Walaja", "Abu Qrenat", "Abu Talul",
                      "Ar\'arat an-Naqab", "Ateer", "al-Atrash", "Bir Hadaj", "Dhahiyah", "Drijat", "Ghazzah", "Hura",
                      "Kukhleh", "Kuseife", "Lakiya", "Makhul", "Mitnan", "Mulada", "Qasr al-Sir", "Rahat", "al-Sayyid",
                      "Shaqib al-Salam", "Tirabin al-Sana", "Tel as-Sabi", "Umm Batin", "Abu Sinan", "Arab al-Aramshe",
                      "Arab al-Subeih", "Arab al-Na\'im", "Arraba", "Basmat Tab\'un", "Beit Jann", "Bi\'ina",
                      "Bir al-Maksur", "Bu\'eine Nujeidat", "Buqei\'a", "Daburiyya", "Ed Dahi", "Deir al-Asad",
                      "Deir Hanna", "Dmeide", "Eilabun", "Ein al-Asad", "Ein Mahil", "Fassuta", "Hamaam", "Hamdon",
                      "Hurfeish", "Hussniyya", "I\'billin", "Iksal", "Ilut", "Jadeidi-Makr", "Jish", "Julis",
                      "Ka\'abiyye-Tabbash-Hajajre", "Kabul", "Kafr Kanna", "Kafr Manda", "Kafr Misr", "Kafr Yasif",
                      "Kamanneh", "Kaukab Abu al-Hija", "Kfar Kama", "Kisra-Sumei", "Maghar", "Majd al-Krum",
                      "Manshiya Zabda", "Mashhad", "Mazra\'a", "Mi\'ilya", "Muqeible", "Nahf", "Na\'ura", "Nazareth",
                      "Nein", "Rameh", "Ras al-Ein", "Rehaniya", "Reineh", "Rumana", "Rumat al-Heib", "Sajur",
                      "Sakhnin",
                      "Sallama", "Sandala", "Sha\'ab", "Shefa-\'Amr", "Sheikh Danun", "Shibli–Umm al-Ghanam", "Sulam",
                      "Suweid Hamira", "Tarshiha", "Tamra City", "Tamra Village", "Tuba-Zangariyye", "Tur\'an", "Uzeir",
                      "Yafa an-Naseriyye", "Yanuh-Jat", "Yarka", "Zarzir", "Bani Suheila", "Beit Hanoun", "Beit Lahiya",
                      "Deir al-Balah", "Jabalia", "Khan Yunis", "Rafah"]

# Cities in Israel ('48 lands)
ISRAELI_CITIES = ["Acre", "Afula", "Arad", "Arraba", "Ashdod", "Ashkelon", "Baqa al-Gharbiyye", "Bat Yam", "Beersheba",
                  "Beit She\'an", "Beit Shemesh", "Bnei Brak", "Dimona", "Eilat", "El\'ad", "Giv\'at Shmuel",
                  "Givatayim",
                  "Hadera", "Haifa", "Herzliya", "Hod HaSharon", "Holon", "Jerusalem", "Kafr Qasim", "Karmiel",
                  "Kfar Saba",
                  "Kfar Yona", "Kiryat Ata", "Kiryat Bialik", "Kiryat Gat", "Kiryat Malakhi", "Kiryat Motzkin",
                  "Kiryat Ono",
                  "Kiryat Shmona", "Kiryat Yam", "Lod", "Ma\'alot-Tarshiha", "Migdal HaEmek",
                  "Modi\'in-Maccabim-Re\'ut",
                  "Nahariya", "Nazareth", "Nesher", "Ness Ziona", "Netanya", "Netivot", "Nof HaGalil", "Ofakim",
                  "Or Akiva",
                  "Or Yehuda", "Petah Tikva", "Qalansawe", "Ra\'anana", "Rahat", "Ramat Gan", "Ramat HaSharon", "Ramla",
                  "Rehovot", "Rishon LeZion", "Rosh HaAyin", "Safed", "Sakhnin", "Sderot", "Shefa-\'Amr", "Tamra",
                  "Tayibe",
                  "Tel Aviv-Yafo", "Tel Aviv", "Tiberias", "Tira", "Tirat Carmel", "Umm al-Fahm", "Yavne",
                  "Yehud-Monosson",
                  "Yokneam Illit"]

PALESTINE_MEMBER_AFFILIATIONS = PALESTINE_IDENTIFIERS + PALESTINIAN_CITIES
ISRAEL_MEMBER_AFFILIATIONS = ISRAEL_IDENTIFIERS + ISRAELI_CITIES

In [23]:
nlp = spacy.load('nl_core_news_lg')

# enumerate spacy subject types
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]

# 27. VB  Verb, base form
# 28. VBD Verb, past tense
# 29. VBG Verb, gerund or present participle
# 30. VBN Verb, past participle
# 31. VBP Verb, non-3rd person singular present
# 32. VBZ Verb, 3rd person singular present
VERBS = ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
ADJECTIVES = ["JJ"]

# Enumerate common words related to death for automated sentence tagging
#FATAL_ADJECTIVES = ["dead", "deceased", "buried", "killed"]
FATAL_ADJECTIVES = ["dood", "overleden", "gestorven", "begraven", "gedood"]
#FATAL_VERBS_PASSIVE = ["die", "decease"]
FATAL_VERBS_PASSIVE = ["sterven", "overlijden"]
#FATAL_VERBS_ACTIVE = ["kill", "murder", "massacre", "shoot", "assassinate", "stab", "slash"]
FATAL_VERBS_ACTIVE = ["doden", "moord", "bloedbad", "schieten", "vermoorden", "steken", "snijden"]
#FATAL_VERBS_ACTIVE_SPECIFIC = ["behead", "slaughter", "execute", "hang"]
FATAL_VERBS_ACTIVE_SPECIFIC = ["onthoofden", "afslachten", "executeren", "ophangen"]
ALL_FATAL_VERBS = FATAL_VERBS_PASSIVE + FATAL_VERBS_ACTIVE + FATAL_VERBS_ACTIVE_SPECIFIC

stop_words = set(stopwords.words('dutch'))
lemmatizer = WordNetLemmatizer()

# Choosing a dataset
sample_size = 1

# Defining the dataset path
results_prefix = "./results"

input_files = os.listdir(results_prefix + '/')

In [4]:
def follow_compound(dep_idx, dependencies_by_governor):
    # Follow compound chain and return descriptors of a dependency
    visited = set([dep_idx])
    current_idx = dep_idx
    descriptors = set()

    found = True
    while found:
        found = False
        for dep in dependencies_by_governor[current_idx]:
            if dep["dep"] == "compound:prt" or dep["dep"] == "compound":
                current_idx = dep["dependent"]
                descriptors.add(dep["dependentGloss"])
                if current_idx not in visited:
                    found = True
                    visited.add(current_idx)
                break
    return descriptors

In [5]:
def investigate_subject(subject, dependencies_by_governor):
    # Investigate all dependencies related to a subject to find as many descriptors as possible
    # Present in a tiered list based on "closeness" to subject

    verbose = False

    subject_descriptors = [set(), set(), set(), 1]
    subject_descriptors[0].add(subject[1])
    # AMOD takes precedence over NMOD? takes precedence over ACL
    for dep in dependencies_by_governor[subj_idx]:
        if (dep["dep"] == "amod"):
            subject_descriptors[0].add(dep["dependentGloss"])
            if verbose:
                print(dep["dep"], dep["dependentGloss"], "\n")

        if (dep["dep"] == "acl" or dep["dep"] == "acl:relcl"):
            subject_descriptors[1].add(dep["dependentGloss"])
            # INVESTIGATE THE SUBJECT AND OBJECT OF DESCRIPTIVE CLAUSE
            if verbose:
                print(dep["dep"], dep["dependentGloss"])
                print(dependencies_by_governor[dep["dependent"]])
                print()
            for double_dep in dependencies_by_governor[dep["dependent"]]:
                # check nsubj and check obj
                if (double_dep["dep"] == "nsubj" or double_dep["dep"] == "nsubj:pass" or double_dep[
                    "dep"] == "nsubj:outer"
                        or double_dep["dep"] == "csubj" or double_dep["dep"] == "csubj:pass" or double_dep[
                            "dep"] == "csubj:outer"
                        or double_dep["dep"] == "obj"):
                    subject_descriptors[2].add(double_dep["dependentGloss"])

        if (dep["dep"] == "nmod" or dep["dep"] == "nmod:npmod" or dep["dep"] == "nmod:tmod" or dep[
            "dep"] == "nmod:poss"):
            subject_descriptors[1].add(dep["dependentGloss"])
            if verbose:
                print(dep["dep"], dep["dependentGloss"], "\n")

        if (dep["dep"] == "advmod"):
            subject_descriptors[1].add(dep["dependentGloss"])  # TODO: CHECK THIS
            if verbose:
                print(dep["dep"], dep["dependentGloss"], "\n")

        if (dep["dep"] == "appos"):
            subject_descriptors[0].add(dep["dependentGloss"])  # TODO:CHECK THIS
            # Look for adjectives for appos also
            # CHECK COMPOUD AND AMOD
            to_add = follow_compound(dep["dependent"], dependencies_by_governor)
            if verbose:
                print(dep["dep"], dep["dependentGloss"], "\n", to_add)
            for thing in to_add:
                subject_descriptors[0].add(thing)
            for double_dep in dependencies_by_governor[dep["dependent"]]:
                if (double_dep["dep"] == "amod"):
                    subject_descriptors[2].add(double_dep["dependentGloss"])

        #         if (dep["dep"] == "ccomp"):
        #             print(dependencies_by_governor[dep["dependent"]])
        #             for double_dep in dependencies_by_governor[dep["dependent"]]:
        #                 if (double_dep["dep"] == "nsubj" or double_dep["dep"] == "nsubj:pass"
        #                      or double_dep["dep"] == "csubj" or double_dep["dep"] == "csubj:pass"):
        #                     subject_descriptors[2].add(double_dep["dependentGloss"])

        if (dep["dep"] == "nummod"):
            try:
                subject_descriptors[3] = locale.atoi(dep["dependentGloss"])
            except:
                subject_descriptors[3] = dep["dependentGloss"]

    return subject_descriptors

In [6]:
def extract_sentences(sentences):
    # extract all sentences in an article
    sentences_text = [None] * len(sentences)
    for sentence in sentences:
        sentence_index = sentence["index"]

        tokens = sentence["tokens"]
        sentence_text = ""
        for token in tokens:
            sentence_text += token["before"] + token["word"] + token["after"]

        sentences_text[sentence_index] = sentence_text
    return sentences_text

In [7]:
from IPython.display import clear_output
def refresh_screen():
    clear_output()
    sleep(0.02)

In [8]:
# load data for annotation
df = pd.read_csv('../data/articles/nos_articles.csv')
# df = pd.read_csv('./data/summary_20231201_livefeeds.csv\')
df['date'] = pd.to_datetime(df['publishedAt'], utc=True)
df.head()

,id,title,description,image,publishedAt,modifiedAt,owner,type,categories,systemTag,keywords,label,duration,url,content,date
0,2519671,Hulporganisaties vrezen voor humanitaire situa...,De twee belangrijkste toevoerlocaties voor noo...,"{'copyright': 'AFP', 'description': None, 'ima...",2024-05-07T21:18:45+0200,2024-05-07T21:18:45+0200,nieuws,article,"[{'name': 'buitenland', 'label': 'Buitenland',...",NaN,"[{'name': 'VN', 'label': 'VN'}, {'name': 'hong...",NaN,NaN,https://nos.nl/collectie/13959/artikel/2519671,AFP\nOorlog Hamas-Israël\nNOS Nieuws\n•\ndinsd...,2024-05-07 19:18:45+00:00
1,2519575,"Israël neemt grensovergang Rafah in, Egypte no...",Het Israëlische leger zegt dat het opereert in...,"{'copyright': 'Reuters', 'description': None, ...",2024-05-07T08:18:19+0200,2024-05-07T13:49:42+0200,nieuws,article,"[{'name': 'buitenland', 'label': 'Buitenland',...",NaN,"[{'name': 'Gaza', 'label': 'Gaza'}]",NaN,NaN,https://nos.nl/collectie/13959/artikel/2519575,Reuters\nOorlog Hamas-Israël\nNOS Nieuws\n•\nd...,2024-05-07 06:18:19+00:00
2,2519464,Israëlische leger roept op tot vertrek burgerb...,Burgers worden opgeroepen om naar Khan Younis ...,"{'copyright': 'Reuters', 'description': None, ...",2024-05-06T09:50:37+0200,2024-05-06T13:41:50+0200,nieuws,article,"[{'name': 'buitenland', 'label': 'Buitenland',...",NaN,"[{'name': 'Israël', 'label': 'Israël'}, {'name...",NaN,NaN,https://nos.nl/collectie/13959/artikel/2519464,Reuters\nOorlog Hamas-Israël\nNOS Nieuws\n•\nm...,2024-05-06 07:50:37+00:00
3,2519427,Blinken kritisch op Israël • Egypte sluit zich...,In dit liveblog houden we je op de hoogte van ...,"{'copyright': 'EPA', 'description': None, 'ima...",2024-05-06T00:24:10+0200,2024-05-12T22:20:24+0200,nieuws,liveblog,"[{'name': 'buitenland', 'label': 'Buitenland',...",NaN,"[{'name': 'Midden Oosten', 'label': 'Midden Oo...",Midden-Oostenblog,NaN,https://nos.nl/collectie/13959/artikel/2519427,Midden-Oostenblog\nOorlog Hamas-Israël\nNOS Ni...,2024-05-05 22:24:10+00:00
4,2518579,Drie militairen gedood door Hamas bij grensove...,In dit liveblog houden we je op de hoogte van ...,"{'copyright': 'AFP', 'description': None, 'ima...",2024-04-29T02:15:20+0200,2024-05-05T22:51:39+0200,nieuws,liveblog,"[{'name': 'buitenland', 'label': 'Buitenland',...",NaN,"[{'name': 'Midden Oosten', 'label': 'Midden Oo...",Midden-Oostenblog,NaN,https://nos.nl/collectie/13959/artikel/2518579,Midden-Oostenblog\nOorlog Hamas-Israël\nNOS Ni...,2024-04-29 00:15:20+00:00


In [20]:
test_article = """Netanyahu dreigt met krachtige actie 'in noorden' • Doden bij aanvallen op kampen Centraal-Gaza

In dit blog houden we je op de hoogte van de laatste ontwikkelingen in het Midden-Oosten.

    Doden bij aanvallen op kampen Centraal-Gaza
    'Israël gebruikte witte fosfor in bewoond gebied'
    '15 doden bij aanval Israël op vluchtelingenkamp'
    Ook Slovenië erkent Palestina

Hier vind je een overzicht van onze belangrijkste verhalen. Hier lees je de betekenis van een aantal veelgebruikte termen. Hier bekijk je onze interactieve special over de machtsstrijd in het Midden-Oosten.

10:50
Israël bereidt zich voor op 'extreem krachtige actie' in Libanon

Premier Netanyahu heeft bij een bezoek aan Noord-Israël gezegd dat zijn land zich op een "extreem krachtige actie" in het noorden voorbereidt. Daarmee kan hij alleen maar een actie tegen Hezbollah bedoelen, de pro-Iraanse Libanese militie die het noorden van Israël dagelijks onder vuur neemt. Hezbollah wil daarmee doorgaan zolang de oorlog in Gaza duurt.

Netanyahu was in Kiryat Shmona, een stadje in Noord-Israël waar voortdurend het luchtalarm afgaat vanwege inkomende raketten en drones. Gisteren braken in Noord-Israël grote natuurbranden uit als gevolg van beschietingen door Hezbollah. "Wie denkt dat hij ons kan raken en dat we met ineengevouwen handen afwachten, maakt een grote vergissing", zei hij.
Ook bij Kiryjat Shmona braken natuurbranden uit door aanvallen van Hezbollah

10:19
'Doden bij aanvallen op kampen Centraal-Gaza'

Bij Israëlische aanvallen op Centraal-Gaza zijn gisteren tientallen doden gevallen. Dat gebeurde onder meer bij aanvallen op twee Palestijnse vluchtelingenkampen.

Het Israëlische leger zei gisteravond dat het in het kamp Bureij met een "precisieaanval" tegen Hamas bezig was. Het is onduidelijk of die al is afgerond. Het leger zei eerder dit jaar dat Centraal-Gaza onder controle was gebracht. Dat gold ook voor Bureij en vluchtelingenkampen daar in de buurt.

De kampen werden in 1948 opgericht voor Palestijnen die Israël al dan niet gedwongen moesten verlaten. In de loop van de tijd veranderde deze kampen in dichtbevolkte stedelijke gebieden.

Al Jazeera hoorde van medische bronnen dat in Centraal-Gaza in de afgelopen 24 uur in totaal 66 mensen zijn omgekomen. Bij dit ziekenhuis kwamen de doden en gewonden binnen:
0:40
Wanhoop rondom mortuarium in Gaza: 'Kinderen zijn doelwit'

09:59
'Israël gebruikte witte fosfor in bewoond gebied'

Human Rights Watch (HRW) zegt dat Israël in de strijd tegen Hezbollah witte fosfor heeft gebruikt. In zeker vijf steden en dorpen zou dit strijdmiddel zijn neergekomen. Volgens de mensenrechtenorganisatie schendt Israël daarmee het internationaal recht, omdat de inzet hiervan in bewoond gebied verboden is. Volgens Israël wordt het middel alleen gebruikt om rookgordijnen op te trekken, niet om burgers te treffen.

Witte fosfor kan gebouwen in brand zetten en zeer ernstige brandwonden, ademhalingsklachten en uitval van organen veroorzaken. Onderzoekers van HRW hebben alleen meldingen over ademhalingsklachten gekregen. Over brandwonden hebben ze niets gehoord.

HRW sprak met acht inwoners van Zuid-Libanon. De organisatie heeft met tientallen foto's en video's vijf steden gelokaliseerd waar het middel neerkwam. Het Libanese ministerie van Gezondheid zegt dat zeker 173 mensen met het middel in aanraking zijn gekomen.

Mensenrechtenorganisatie Amnesty International beschuldigde Israël in oktober al van het inzetten van witte fosfor. Inwoners van het dorp Dhayra vertelden de NOS wat hen overkwam:
1:52
'Ik moest overgeven en kreeg geen zuurstof meer'

    Amnesty: inzet witte fosfor Israël in Libanon onderzoeken als oorlogsmisdrijf

08:51
Vuurgevecht bij Amerikaanse ambassade in Beiroet

Bij de Amerikaanse ambassade in Beiroet heeft een vuurgevecht plaatsgevonden. Volgens het Libanese leger opende een gewapende man het vuur bij de ingang van het ambassadeterrein. Libanese militairen schoten terug, waardoor de man gewond is geraakt. Hij zou uit Syrië komen. Hij is aangehouden en naar het ziekenhuis gebracht.

Het vuurgevecht heeft volgens lokale media bijna een half uur geduurd. De Amerikaanse ambassade laat weten dat iedereen veilig is.

08:22
'15 doden bij aanval Israël op vluchtelingenkamp'

Israël heeft gisteren vanuit de lucht en over de grond een Palestijnse vluchtelingenkamp in Centraal-Gaza aangevallen. Daarbij zijn volgens de lokale autoriteiten zeker vijftien doden gevallen. Het Israëlische leger was het vluchtelingenkamp Bureij begin dit jaar ook al binnengevallen, maar trok zich daarna weer terug.

Het Israëlische leger zegt dat een gebouw van de VN-hulporganisatie UNRWA is gebombardeerd omdat daar een post van Hamas in zou zitten. Bij Al Jazeera is te zien dat gewonden bij een ziekenhuis worden afgeleverd. In het mortuarium liggen de lichamen van de leden van een familie, onder wie een jong meisje. De verslaggever verwacht dat de aanvallen doorgaan en er meer slachtoffers gaan vallen.

Al Jazeera zegt dat ook het vluchtelingenkamp Maghazi, vlak bij Bureij, door Israël is aangevallen. Daar zouden zeker acht mensen zijn omgekomen.
Een vrouw bij een ziekenhuis waar slachtoffers van de aanval op Bureij naartoe zijn gebracht

07:00
Bemiddelaars staakt-het-vuren wachten op antwoord Hamas

De bemiddelaars voor een staakt-het-vuren hebben nog altijd geen officiële reactie van Hamas op het Israëlische voorstel dat vrijdag door de Amerikaanse president Biden is gepresenteerd. "We wachten op het antwoord van Hamas", zei de Amerikaanse nationaal veiligheidsadviseur Jake Sullivan gisteravond.

Volgens hem sluit president Biden niet uit dat Hamas de oorlog wil rekken, maar hoopt hij dat Hamas inziet dat het voorstel de beste manier is om de oorlog te stoppen, de gegijzelden terug te brengen en de humanitaire hulp op gang te brengen.
Standpunt Israël

Ook Israël heeft nog steeds niet met het plan ingestemd. Premier Netanyahu noemde het voorstel zelfs kansloos, zolang niet alle Israëlische oorlogsdoelen waren gehaald. Toch ontkent Sullivan met klem dat Israël het voorstel afwijst. "Ze hebben opnieuw bevestigd dat ze het accepteren, dat hebben ze eerder gedaan en dat doen ze nog. De bal ligt bij Hamas."

Biden houdt het erop dat Netanyahu eerst een probleem in eigen huis moet oplossen. Belangrijke, uiterst rechtse leden dreigen zijn kabinet te laten vallen, als hij het voorstel dat nu op tafel ligt accepteert.
Positief

Hamas reageerde vrijdag in een eerste reactie positief. Het wachten is op het officiële standpunt. De leiding van Hamas houdt zich schuil, de communicatie met de Hamas-leiders loopt via bemiddelaar Qatar.

Een Hamas-leider in Beiroet zei gisteren dat Hamas eist dat in het uiteindelijke akkoord duidelijk omschreven moet zijn dat er een permanent staakt-het-vuren komt en dat Israël zich volledig uit Gaza terugtrekt.
Nationaal veiligheidsadviseur Jake Sullivan

    'Oorlog moet stoppen': Biden presenteert nieuw plan Israël voor wapenstilstand

gisteren, 22:05
Slovenië erkent Palestijnse staat

Slovenië gaat de Palestijnse staat erkennen. Het Sloveense parlement stemde na een lange sessie, waarbij de uiteindelijke stemming werd geboycot door de centrumrechtse oppositie, voor de erkenning.

Met de erkenning volgt Slovenië het voorbeeld van Noorwegen, Spanje en Ierland. Die landen maakten bijna twee weken geleden bekend dat ze de Palestijnse staat gingen erkennen. Kort na de aankondiging maakte Israël bekend zijn ambassadeurs terug te roepen uit die landen. Het merendeel van de lidstaten van de VN erkent de staat Palestina, maar dit zijn vooral landen in het mondiale zuiden.

Nederland erkent de Palestijnse staat niet. Het ministerie van Buitenlandse Zaken zei in een reactie op de erkenning door Noorwegen, Spanje en Ierland dat Nederland voor een tweestatenoplossing is, "waarbij een veilig Israël en een onafhankelijke en levensvatbare Palestijnse staat vreedzaam naast elkaar bestaan". Tegelijk moet erkenning van een Palestijnse staat "onderdeel zijn van het politieke proces naar een tweestatenoplossing. Eenzijdige erkenning brengt een oplossing op dit moment niet dichterbij: dat vraagt om een zorgvuldige afweging", aldus het ministerie.
Een Palestijnse vlag wappert naast een Sloveense en Europese vlag op het overheidsgebouw in Ljubljana

gisteren, 20:25
Hamas: alleen deal met duidelijke afspraken over permanent staakt-het-vuren

Hamas kan enkel akkoord gaan met een voorstel voor een staakt-het-vuren, als daarin duidelijke afspraken staan over een permanent staakt-het-vuren en een volledige terugtrekking van het Israëlische leger uit de Gazastrook. Dat zei Hamas-kopstuk Osama Hamdan in een persconferentie in Beiroet.

Hamdan zei verder de bemiddelaars die betrokken zijn bij gesprekken over een deal te hebben gevraagd dat Israël zich "helder" committeert aan afspraken over die permanente gevechtspauze en volledige terugtrekking; die twee punten moeten volgens Hamas naar eigen zeggen terugkeren in een akkoord.
Osama Hamdan op archiefbeeld uit 2021

gisteren, 17:20
Biden denkt dat Netanyahu oorlog rekt voor politiek gewin

De Amerikaanse president Biden denkt dat de Israëlische premier Netanyahu de oorlog in Gaza laat voortduren voor zijn eigen politieke positie. "Er is alle reden voor mensen om die conclusie te trekken", zegt hij in een interview met het tijdschrift Time. Ook veel anti-regeringsdemonstranten denken dat Netanyahu een deal met Hamas blokkeert om zelf aan de macht te blijven.

Gevraagd naar de mogelijkheid dat Israël oorlogsmisdaden pleegt in Gaza zegt Biden dat dat onzeker is. "Maar een ding staat als een paal boven water: de mensen in Gaza, de Palestijnen, hebben enorm te lijden onder een gebrek aan voedsel, water, medicijnen, et cetera. En heel veel onschuldige mensen zijn gedood." Daarbij wijst hij ook naar Hamas en de terreuraanslag op 7 oktober. "Wat zij toen hebben gedaan, gaat verder dan wat ik ooit heb gezien. En ik heb een boel meegemaakt." Ook vindt hij dat Hamas een staakt-het-vuren tegenhoudt.

Sinds het uitbreken van de oorlog is de relatie tussen de VS en Israël onder druk komen te staan. De VS is van oudsher Israëls belangrijkste bondgenoot. Hoewel Amerikaanse gezagsdragers blijven benadrukken dat ze vierkant achter Israël staan, klinkt er ook stevige kritiek. Zo zei Biden in maart dat Netanyahu meer kwaad dan goed doet voor Israël en noemde de Democratische leider in de Senaat de Israëlische premier een obstakel voor vrede.

    Protest tegen Netanyahu groeit: 'We willen nu nieuwe verkiezingen'

gisteren, 15:56
Tientallen arrestaties in Den Haag bij pro-Palestijns protest

Bij een pro-Palestijns protest in het centrum van Den Haag zijn tientallen actievoerders aangehouden door de politie. Ze protesteerden bij het gebouw van universiteitskoepel Universiteiten van Nederland aan de Lange Houtstraat en reageerden niet op oproepen van de politie om de betoging te beëindigen.

De betogers willen dat de universiteiten vanwege de oorlog in de Gazastrook hun banden met Israëlische instellingen verbreken. Aan het pand hingen ze spandoeken op met teksten als "Nederlandse universiteiten medeplichtig aan genocide" en "Boycott en divest". Ook hielden ze de entree bezet. Ze kwamen het kantoor zelf niet binnen omdat daarvoor een toegangspasje nodig is, meldt persbureau ANP.

Een aantal activisten was via een zelf meegebrachte ladder op een overkapping boven de ingang van het pand gaan staan. Zij werden met een hoogwerker van de brandweer naar beneden gehaald. Tientallen medestanders waren toen al een voor een in een stadsbus gezet. Rond 15.00 uur reed de bus weg.

De meesten zijn aangehouden voor lokaalvredebreuk. Ook waren er volgens de politie een aantal overtredingen van de Wet openbare manifestaties en is er iemand opgepakt vanwege belediging van agenten.
0:43
Pro-Palestijns protest bij universiteitskoepel in Den Haag

gisteren, 14:45
Coalitiepartner Netanyahu steunt voorstel staakt-het-vuren

De ultraorthodoxe Shas-partij, de grootste coalitiepartner van premier Netanyahu, steunt het door Biden gepresenteerde plan voor een staakt-het-vuren "volledig". Het toont de verdeeldheid in de Israëlische regering, waar twee radicaal-rechtse ministers zich juist tegen het plan verzetten.

Netanyahu zei eerder dat het plan van Biden niet volledig overeenkomt met het voorstel dat Israël heeft ingediend. Bemiddelaar Qatar spreekt dat tegen, en zegt dat het een voorstel van Israël heeft ontvangen dat wel degelijk overeenkomt met wat Biden schetste. Qatar heeft het Israëlische voorstel bij Hamas neergelegd.

De woordvoerder van het Qatarese ministerie van Buitenlandse Zaken zegt dat met dit voorstel de posities van beide partijen elkaar naderen.

gisteren, 13:33
Qatar over vredesplan: 'We wachten op duidelijk Israëlisch standpunt'

Qatar, bemiddelaar tussen Israël en Hamas, zegt van beide nog geen "concrete goedkeuring" te hebben ontvangen op het voorstel van de Amerikaanse president Biden. Beide partijen moeten een duidelijke positie hebben om tot een wapenstilstand te komen, zegt de woordvoerder van het Qatarese ministerie van Buitenlandse Zaken, Majed Al-Ansari.

"We wachten op een duidelijk Israëlisch standpunt dat de hele regering vertegenwoordigt als reactie op het Amerikaanse Gaza-voorstel", aldus Al-Ansari.

Vrijdag presenteerde Biden een voorstel voor een staakt-het-vuren, waaronder Hamas gefaseerd de gijzelaars zou vrijlaten in ruil voor Palestijnse gevangenen en terugtrekking van het Israëlische leger uit Gaza. Volgens Biden ging het om een "Israëlisch" voorstel, terwijl premier Netanyahu het voorstel onvolledig noemde en "kansloos" zolang niet alle Israëlische oorlogsdoelen zijn bereikt.

Twee radicaal-rechtse Israëlische ministers keerden zich tegen het plan. Minister van Nationale Veiligheid Ben-Gvir dreigt het kabinet ten val te brengen als Netanyahu een akkoord sluit zonder dat Hamas is vernietigd.

Ben-Gvir noemde het vredesplan van Biden roekeloos:
0:20
Israëlische minister Ben-Gvir noemt vredesplan 'roekeloos' en dreigt met kabinetsval

gisteren, 11:40
Oxfam: Israël maakt hulpverlening Gaza 'vrijwel onmogelijk'

De aanhoudende Israëlische bombardementen in de Gazastrook en het "opzettelijke tegenhouden van humanitaire hulp" maken het voor hulporganisaties vrijwel onmogelijk om de burgerbevolking te bereiken. Dat zegt de internationale hulporganisatie Oxfam. "Hongersnood komt daardoor dichterbij," aldus Oxfam.

Met de Israëlische inname en sluiting van de Rafah-grenspost tussen Egypte en Gaza op 7 mei is de hoeveelheid hulp die Gaza binnenkomt drastisch afgenomen. Meer dan 2000 vrachtwagens staan vast aan de Egyptische kant van de grens. Volgens Oxfam sterven er nu meer kinderen door honger en ziekte.

Volgens de Palestijnse hulporganisatie Al Mezan zijn het afgelopen weekend twee kinderen als gevolg van ondervoeding en gebrek aan adequate zorg overleden.

Een Israëlisch rapport stelde vorige maand dat Israël voldoende voedsel toelaat en dat "voedselonzekerheid" wordt veroorzaakt door distributieproblemen in de Gazastrook zelf. Oxfam zegt dat veel van de vrachtwagens die door Israël worden toegelaten van commerciële partijen komen, dus niet van hulporganisaties. Deze zouden vooral niet-voedzame producten vervoeren zoals energiedrankjes, koekjes en chocolade.
Een Palestijnse jongen sleept met twee jerrycans water in het noorden van de Gazastrook

gisteren, 11:15
Israël: zeker 43 gijzelaars niet meer in leven

Meer van een derde van de overgebleven gijzelaars in Gaza is niet meer in leven, volgens een nieuwe telling van de Israëlische regering. Hamas zou nog 120 gijzelaars vasthouden, van wie er volgens Israël zeker 43 dood zijn.

Israël baseert dit op informatie verkregen via inlichtingen, videomateriaal en forensische analyse. Gisteren maakte Israël de namen van vier omgekomen gijzelaars bekend, zonder informatie te verschaffen over de doodsoorzaak. Hamas zei vorige maand dat een andere gijzelaar was overleden aan zijn verwondingen opgelopen in een Israëlische luchtaanval.

Hamas gijzelde bij de aanslag op 7 oktober rond de 250 mensen. 105 werden in november vrijgelaten als onderdeel van een staakt-het-vuren. Drie weken geleden haalde Israël de lichamen terug van drie dode gijzelaars.

De vrijlating van de overgebleven gijzelaars is een kernpunt van de onderhandelingen tussen Israël en Hamas over een wapenstilstand. Vrijdag presenteerde de Amerikaanse president Biden een voorstel voor een bestand dat volgens hem uit de koker van Israël kwam, maar premier Netanyahu noemde Bidens omschrijving ervan "onvolledig". Daarop reageerde een woordvoerder van het Witte Huis dat het voorstel van Biden wel degelijk overeenkwam met het Israëlische plan.
Een bord in Tel Aviv dat verwijst naar de gijzelaars in Gaza

gisteren, 10:32
Doden bij Israëlische luchtaanvallen in vluchtelingenkamp Gaza

Bij een Israëlische luchtaanval op het vluchtelingenkamp Bureij in Centraal-Gaza zijn vannacht drie Palestijnen omgekomen, meldt het Palestijnse persbureau Wafa. Het Israëlische leger heeft sinds zondag meerdere luchtaanvallen uitgevoerd op Bureij, waarbij in totaal veertien Palestijnen zouden zijn omgekomen.

Het leger zelf heeft geen informatie gedeeld over aanvallen in Bureij. Nieuwszender Al Jazeera meldt dat er vannacht en vanochtend ook meerdere doden zijn gevallen bij Israëlische aanvallen op andere plekken in de Gazastrook, waaronder drie in Gaza-Stad en zeven in Deir Al-Balah.

Beelden van Wafa van een van de luchtaanvallen in Bureij, gisteren:
0:17
Israëlische luchtaanval op gebouw bij vluchtelingenkamp in Gaza

gisteren, 09:33
Twee Palestijnen doodgeschoten op Westelijke Jordaanoever

Het Israëlische leger heeft twee Palestijnen doodgeschoten op de Westelijke Jordaanoever. Volgens het leger waren de twee mannen, afkomstig uit het Tulkarem-vluchtelingenkamp, op weg om een aanslag te plegen op Israëlische nederzettingen. In een bericht op X zegt het leger dat de mannen, met geweren bij zich, in een hinderlaag liepen.

Het Palestijnse persbureau Wafa meldt dat het leger het vuur op de auto van de mannen opende bij een checkpoint in de buurt van een Israëlische nederzetting en verhinderde dat ambulances de slachtoffers konden bereiken.

Daarnaast werden volgens Wafa gisteren drie Palestijnen gedood bij een Israëlische inval in de Palestijnse stad Nablus. Sinds 7 oktober zijn er volgens cijfers van de VN op de Westoever meer dan 500 Palestijnen gedood door het Israëlische leger en kolonisten, en 12 Israëliërs bij aanslagen van Palestijnen.
Inval van het Israëlische leger in Nablus op 3 juni

gisteren, 04:29
Bosbranden in Noord-Israël na raketaanvallen Hezbollah

In het noorden van Israël woeden bosbranden als gevolg van raketten die vanuit Libanon zijn gelanceerd door de militante groepering Hezbollah, meldt persbureau Reuters. Het vuur verspreidt zich snel door het hete en droge weer.

Het Israëlische leger heeft materialen en militairen gestuurd om de branden onder controle te krijgen. Zes militaire reservisten zijn lichtgewond geraakt.
Branden in het noorden van Israël na luchtaanvallen vanuit het zuiden van Libanon

"De troepen hebben controle gekregen over de locaties van de brand, en in dit stadium is er geen mensenleven in gevaar", meldt het leger. Bewoners die zich nog in het gebied van de branden bevinden, zijn opgeroepen om te vertrekken. Veel Israëliërs die vlak bij de grens met Libanon woonden zijn maanden geleden al geëvacueerd vanwege de escalerende gevechten tussen Israël en Hezbollah.

De branden besloegen gisteren volgens de Israëlische autoriteiten al honderden hectares. De nationale brandweer was tot laat in de nacht aan het werk, onder meer in de stad Kiryat Shmona.

gisteren, 01:53
VS zoekt steun VN-Veiligheidsraad voor voorstel wapenstilstand

De Verenigde Staten willen dat de Veiligheidsraad van de Verenigde Naties een resolutie aanneemt om het plan te steunen van de Amerikaanse president Joe Biden voor een wapenstilstand in Gaza, in ruil voor de vrijlating van alle gijzelaars door Hamas. Dat heeft de Amerikaanse VN-ambassadeur Linda Thomas-Greenfield gezegd. "Talloze leiders en regeringen, ook in de regio, hebben dit plan gesteund en we roepen de Veiligheidsraad op om zich bij hen aan te sluiten om het akkoord zonder uitstel en zonder verdere voorwaarden uit te voeren", stelde Thomas-Greenfield in een verklaring.

Biden presenteerde vrijdag een "routekaart voor een duurzaam staakt-het-vuren", die volgens de president van Israël kwam. In dit plan staat een tijdelijk "volledig staakt-het-vuren" en terugtrekking van Israëlische militairen uit "bevolkte gebieden in Gaza". Daarna moet Israël zich volledig terugtrekken uit Gaza en moeten alle gevangenen worden vrijgelaten.

maandag, 22:10
Houthi's vuren raket af op Eilat, neergehaald door Israël

De Houthi's hebben vanuit Jemen een ballistische raket afgevuurd op de Israëlische stad Eilat. Het Israëlische leger zegt de raket boven de stad in het uiterste zuiden van Israël uit de lucht te hebben geschoten. Op beelden op sociale media zijn rookpluimen te zien in de buurt van de stad.

De door Iran gesteunde Houthi's vuren vaker rakketten en drones af op Eilat, meestal zonder succes. De terreurgroep zegt dat te doen uit solidariteit met de Palestijnen. Ook zijn geregeld westerse schepen in de Rode Zee doelwit van de Houthi's. Sinds november mijden veel schepen ook de Rode Zee.

Via luchtaanvallen op doelen in Jemen proberen westerse eenheden onder leiding van de VS en het VK geregeld de Houthi's te treffen en te verzwakken.

maandag, 19:54
Vier Israëlische gijzelaars overleden in Gazastrook

Het Israëlische leger heeft bevestigd dat nog eens vier gijzelaars die in de Gazastrook werden vastgehouden door Hamas, zijn overleden. Onder de doden zijn drie mannen van boven de 80 jaar, die in december nog te zien waren in een door Hamas vrijgegeven video. "Laat ons hier niet oud worden", zeiden Amiram Cooper, Yoram Metzger en Haim Peri destijds zichtbaar verzwakt. Het vierde slachtoffer is Nadav Popplewell, een 51-jarige man, die ook een Brits paspoort had. Ook hij was eerder te zien in een video van Hamas.

Volgens een woordvoerder van het leger zijn de mannen overleden in Khan Younis, in het zuiden van de Gazastrook. De oorzaak van hun dood is niet bekend. Israël zegt de zaak te onderzoeken en alle vragen te zullen beantwoorden. Vorige maand zei Hamas dat Popplewell was omgekomen nadat hij zwaargewond zou zijn geraakt bij een Israëlische luchtaanval, maar dat werd tot nu toe nooit bevestigd.

Naar verluidt worden er nog ongeveer 130 gijzelaars door Hamas vastgehouden in de Gazastrook. Onduidelijk is hoeveel er nog in leven zijn.
Met de klok mee vanaf linksboven: Chaim Peri, Amiram Cooper, Nadav Popplewell en Yoram Metzger.

maandag, 19:08
Witte Huis ontkent verschillen tussen plan Biden en voorstel Netanyahu

Het Witte Huis ontkent dat er verschillen zijn tussen het plan voor een staakt-het-vuren dat president Biden vrijdag presenteerde en het voorstel dat Israël heeft ingediend. Vandaag zei de Israëlische premier Netanyahu dat het plan van Biden niet volledig overeenkomt met de plannen van Israël.

"We zijn ervan overtuigd dat dit een nauwkeurige weerspiegeling is van het voorstel waar we met de Israëli's aan hebben samengewerkt", zegt de Amerikaanse veiligheidsadviseur John Kirby. "Dus ik zie geen verschillen. Dit is een Israëlisch voorstel. De minister van Buitenlandse Zaken heeft dat zelf bevestigd en ook de premier heeft bevestigd dat dit hun voorstel is."

Een dag na de presentatie van Biden noemde Netanyahu het voorstel "kansloos" zolang niet alle Israëlische oorlogsdoelen zijn bereikt. Dat veroorzaakte verwarring, omdat Biden had gezegd dat het een Israëlisch plan was. Verschillende Israëlische functionarissen bevestigden dit weekend de uitspraken van Biden in de Britse krant The Times.

Kirby benadrukt dat het plan niet naar buiten is gebracht om de druk op Netanyahu op te voeren. De premier staat onder druk omdat extreemrechtse leden van zijn kabinet het niet eens zijn met dit voorstel. Ze dreigen hun steun aan de regering in te trekken, omdat ze willen dat Hamas volledig wordt vernietigd en alle gijzelaars worden vrijgelaten.

    'Oorlog moet stoppen': Biden presenteert nieuw plan Israël voor wapenstilstand

maandag, 17:29
Correspondent over uitspraken Netanyahu, die het voorstel van Biden onvolledig noemt
Correspondent Nasrah Habiballah:

"De toespraak van Biden waarin hij het voorstel dat uit Israëlische koker zou komen presenteerde, maakte in Israël zelf best wat los. Het leek af te wijken van de positie die Netanyahu de hele oorlog al vasthield, namelijk dat een permanent staakt-het-vuren pas een optie is als Hamas is vernietigd.

Maar dat beeld nuanceerde Netanyahu snel met een statement, en daar doet hij nu dus nog een schepje bovenop. Want al snel kreeg hij veel kritiek vanuit de ultrarechtse partijen binnen zijn coalitie. Maar met de uitspraken van Netanyahu kan je je afvragen wat er nu dan daadwerkelijk is veranderd aan hoe de strijdende partijen hierin staan. En het gevoel dat men met de toespraak van Biden kreeg dat een einde aan de oorlog wellicht in zicht is, daar lijkt nog maar weinig van over."

maandag, 16:54
'Israël gaat strafrechtelijk onderzoek doen naar dood tientallen Gazaanse gevangenen'

Het Israëlische leger gaat een strafrechtelijk onderzoek doen naar de dood van 36 Gazaanse gevangenen die omkwamen in de beruchte militaire gevangenis Sde Teiman, in de Negev-woestijn. De gevangenis staat op een militaire basis en staat bekend als het Israëlische Guantánamo Bay.

Verschillende media schreven al eerder over de erbarmelijke omstandigheden waarin Palestijnse gevangenen er leven. Zo zouden gevangenen er constant geboeid en geblinddoekt zijn en worden ze regelmatig zwaar mishandeld.

Het leger onderzoekt ook twaalf andere sterfgevallen, schrijft de Israëlische krant Haaretz. In die gevallen gaat het bijvoorbeeld om gedetineerden die in Israël onderweg naar de gevangenis overleden. Ook betreft het Palestijnen die in Gaza overleden, bijvoorbeeld omdat ze een bevel van een militair negeerden of Israëlische troepen naderden bij het grenshek. Bronnen binnen het Israëlische leger zeggen tegen Haaretz dat er nog geen verdachten zijn gearresteerd en dat er nog niemand vervolgd gaat worden.

Sinds het begin van de oorlog worden de meeste Palestijnse gevangenen door Israël in de Gazastrook gevangengehouden. Alleen mensen van wie militairen denken dat ze nuttige informatie hebben worden overgebracht naar gevangenissen in Israël. Naar verluidt zijn dat er tot nu enkele duizenden.

Palestijnse gevangenen worden internationaal beschouwd als krijgsgevangenen en moeten worden beschermd door het humanitair oorlogsrecht.

Eerder maakte correspondent Nasrah Habiballah deze explainer over waarom er zo veel Palestijnen langdurig worden vastgezet door Israël:
10:01
Administratieve detentie: zo worden Palestijnen langdurig vastgezet door Israël

maandag, 16:02
VN: meer dan helft gebouwen Gaza vernietigd of beschadigd

Sinds de oorlog op 7 oktober begon is meer dan 55 procent van de gebouwen in de Gazastrook vernietigd of beschadigd. Dat zegt de VN op basis van analyses van satellietbeelden. Zo'n 137.000 gebouwen zijn getroffen door de oorlog.

De schatting is gebaseerd op een satellietbeeld dat gemaakt is op 3 mei. Het beeld is vergeleken met beelden uit mei en september vorig jaar. Ook satellietbeeld van november en begin dit jaar speelde een rol in het onderzoek.

Volgens de VN zijn bijna alle gebouwen die zijn vernietigd of beschadigd woonhuizen. Bijna 17.000 gebouwen zijn ernstig beschadigd in Gaza.

Vorige week publiceerden we deze video waarin Ikram al-Ham terugkeerde naar Khan Younis, dat een spookstad geworden is:
1:17
Terug naar het verwoeste Khan Younis: 'Het is een spookstad, doodeng'

maandag, 15:14
Netanyahu: het voorstel voor beëindiging oorlog dat Biden presenteerde was onvolledig

Volgens de Israëlische premier Netanyahu was de Amerikaanse president Biden niet volledig toen die vrijdag sprak over een plan voor een staakt-het-vuren. Biden had het in een toespraak over een "Israëlisch voorstel" dat zou moeten leiden tot het einde van de oorlog. Een dag later temperde Netanyahu de hoop op de totstandkoming van zo'n bestand echter: hij noemde het "kansloos" zolang niet alle Israëlische oorlogsdoelen zijn bereikt. Dat leidde tot verwarring, want Biden had gezegd dat het plan uit de koker van Israël zelf was gekomen.

In een reactie stelt de Israëlische premier nu dat het plan dat Biden presenteerde niet volledig overeenkomt met het voorstel dat Israël heeft ingediend. "Het voorstel dat Biden presenteerde is onvolledig", citeert het Israëlische nieuwskanaal Channel 12 de premier. "De oorlog zal stoppen om de gijzelaars terug te brengen, en dan zullen we pas verder praten. Er zijn nog andere details die de Amerikaanse president niet aan het publiek heeft gepresenteerd."

Volgens de toespraak van Biden bestaat het plan uit drie fases. De eerste fase zou een wapenstilstand omvatten, net als de terugkeer van enkele gijzelaars die door Hamas worden vastgehouden. Daarna zouden de partijen onderhandelen over een onbepaalde stopzetting van de vijandelijkheden voor een tweede fase. Daarin zouden de resterende gevangenen vrijgelaten worden, zei Biden. Israël heeft echter altijd volgehouden dat Hamas eerst verslagen moet worden voordat het leger zich zou terugtrekken. Die voorwaarde benoemde Biden niet in zijn toespraak.
Ministers dreigen kabinet ten val te brengen

Het door Biden gepresenteerde plan heeft ertoe geleid dat de spanning binnen de Israëlische regering, die toch al groot was, nog meer is toegenomen. Twee belangrijke Israëlische ministers keerden zich gisteren tegen het plan. Minister van Nationale Veiligheid Ben-Gvir dreigt het kabinet ten val te brengen als Netanyahu een akkoord sluit zonder dat Hamas is vernietigd. Zo'n "onbezonnen" akkoord zou volgens hem een overwinning voor terrorisme betekenen en een bedreiging voor de veiligheid van het land.

maandag, 10:59
Egypte hekelt aanwezigheid Israël bij grens Rafah

Egypte eist dat Israël vertrekt uit de Palestijnse kant van de grensovergang bij Rafah, de zogenoemde Philadelphi Corridor. Het Israëlische leger nam de gehele grensovergang met Gaza vorige maand over bij het offensief langs de zuidelijke grens, tot grote ergernis van Egypte.

"Het is moeilijk voor de grensovergang bij Rafah om te blijven opereren zonder een Palestijnse aanwezigheid", zei de Egyptische minister van Buitenlandse Zaken vanochtend op een persconferentie met zijn Spaanse collega in Madrid.

Kort nadat Israël de controle over had genomen, dreigde Egypte al dat het zou stoppen met de samenwerking met Israël bij de bufferzone. Die is sinds de oorlog een belangrijke ader voor de levering van hulp naar de Gazastrook maar ook voor evacuaties.
'Zuurstoflijn Hamas'

Een van de doelen van het Israëlische leger bij de grondoffensief in Rafah was om die grensovergang over te nemen. "De Philadelphi Corridor diende als zuurstoflijn voor Hamas, die die regelmatig gebruikte om wapens naar het gebied van de Gazastrook te smokkelen", zei de Israëlische legerwoordvoerder Daniel Hagari. Egypte heeft dat weersproken.

De Philadelphi Corridor is een honderd meter brede strook grond langs de 14 kilometer lange grens tussen Egypte en de Gazastrook. Daar ligt ook de grensovergang bij Rafah. Egypte en Israël kwamen in 1979 overeen dat dit een bufferzone zou worden, bedoeld om de smokkel van wapens en mensen tegen te gaan.

maandag, 09:08
Lichaam vermiste Israëlier gevonden in kibboets

Acht maanden na de aanslag van 7 oktober is het lichaam van een 35-jarige vermiste Israëliër geïdentificeerd. Het lichaam van Dolev Yehud werd gevonden in een kibboets die terreurorganisatie Hamas aanviel.

Aanvankelijk werd aangenomen dat de man werd gegijzeld in de Gazastrook. Maar omdat er later werd getwijfeld of hij daadwerkelijk was meegenomen, werden de stoffelijke resten onderzocht die eerder bij de kibboets waren gevonden. Zo kwamen er de afgelopen maanden geen signalen uit Gaza dat Yehud daar vastzat. Met hulp van forensische experts is zijn lichaam alsnog geïdentificeerd.

Yehud was hulpverlener en woonde in kibboets Nir Oz. Tijdens de aanslag op 7 oktober ging hij naar buiten om mensen te helpen. Zijn hoogzwangere vrouw en hun drie kinderen bleven in hun schuilplaats.
Dolev Yehud

maandag, 07:23
Palestijnen: zeker 12 doden bij nachtelijke aanvallen

Het Palestijnse persbureau Wafa meldt dat er vannacht bij Israëlische aanvallen op Khan Younis en Rafah in het zuiden van Gaza zeker twaalf mensen zijn gedood. In Khan Younis zou een huis zijn geraakt waar een familie woonde. Onder de dodelijke slachtoffers zijn drie kinderen, schrijft Wafa.

Verder wordt een aanval gemeld die gisteravond plaatsvond in de vluchtelingenkampen Bureij en Nuseirat in centraal Gaza. Tien mensen kwamen daarbij om, aldus het Palestijnse persbureau.

maandag, 04:00
Doden na Israëlische aanval bij Aleppo

Bij een Israëlische luchtaanval op een fabriek in de buurt van Aleppo in Syrië zijn meerdere doden gevallen, dat melden Syrische staatsmedia. Volgens het Syrische Observatorium voor de Mensenrechten, dat vanuit Londen opereert, vielen er twaalf dodelijke slachtoffers. Israël heeft nog niet gereageerd.

Afgelopen jaren heeft Israël honderden aanvallen op Syrië uitgevoerd, vooral tegen de pro-Iraanse Libanese groepering Hezbollah en Iraanse doelwitten. Daarmee wil het naar eigen zeggen Iran verhinderen geavanceerde wapens te leveren aan Hezbollah of een eigen militaire aanwezigheid in Syrië op te bouwen. Sinds Iran de Syrische president Assad steunt in de burgeroorlog, groeit de Iraanse invloed in Syrië.

Bekijk in deze special hoe de machtsverhoudingen in Syrië liggen:
Sla over

maandag, 03:17
Israël onderzoekt alternatief voor het besturen van Gaza door Hamas

De Israëlische minister van Defensie Gallant zegt dat de legerleiding op zoek is naar een alternatief voor het bestuur van Gaza door Hamas. Dat moet al tijdens het proces naar een permanent-staakt-het vuren het bestuur over Gaza van Hamas overnemen.

In de tussentijd blijft Israël streven naar de volledige vernietiging van Hamas. "We isoleren een gebied, halen de mensen van Hamas daar weg en installeren er krachten die een alternatief bestuur kunnen vormen", zei Gallant. Hij zei niet welke krachten dat zouden kunnen zijn.

President Biden presenteerde vrijdag de hoofdlijnen van een route naar een permanent-staakt-het vuren in Gaza. Hamas is al sinds 2007 aan de macht in Gaza. Niets wijst erop dat de militante beweging bereid is zijn macht op te geven.
Hamas-leider Yahya Sinwar in Gaza-Stad in 2021
"""

In [24]:
doc = nlp(test_article)
sentences = [sent.text for sent in doc.sents]
sentences

["Netanyahu dreigt met krachtige actie 'in noorden'",
 '• Doden bij aanvallen op kampen Centraal-Gaza\n\n',
 'In dit blog houden we je op de hoogte van de laatste ontwikkelingen in het Midden-Oosten.\n\n    ',
 "Doden bij aanvallen op kampen Centraal-Gaza\n    'Israël gebruikte witte fosfor in bewoond gebied'\n    '15 doden bij aanval Israël op vluchtelingenkamp'\n    ",
 'Ook Slovenië erkent Palestina\n\n',
 'Hier vind je een overzicht van onze belangrijkste verhalen.',
 'Hier lees je de betekenis van een aantal veelgebruikte termen.',
 'Hier bekijk je onze interactieve special over de machtsstrijd in het Midden-Oosten.',
 '\n\n10:50\nIsraël bereidt zich voor op \'extreem krachtige actie\' in Libanon\n\nPremier Netanyahu heeft bij een bezoek aan Noord-Israël gezegd dat zijn land zich op een "extreem krachtige actie" in het noorden voorbereidt.',
 'Daarmee kan hij alleen maar een actie tegen Hezbollah bedoelen, de pro-Iraanse Libanese militie die het noorden van Israël dagelijks onder 

In [28]:
for sentence in sentences[1:2]:
    print(sentence)
    # sentence_index = sentence["index"]

#     tokens = sentence["tokens"]
#     dependencies = sentence["basicDependencies"]
#     sentence_text = ""
#     for token in tokens:
#         sentence_text += token["before"] + token["word"] + token["after"]
        
#     #print(sentence_text)

#     # create a data structure that maps governor dep_idx to dependencies
#     dependencies_by_governor = [[] for i in range(len(tokens) + 1)]
#     for dep in dependencies:
#         gov_idx = dep["governor"]
#         dependencies_by_governor[gov_idx].append(dep)

#     # create data structure for tokens
#     tokens_by_idx = {}
#     for token in tokens:
#         tokens_by_idx[token["index"]] = token

#     prepositional_information = None
#     for token in tokens:
#         pos = token["pos"]
#         word = token["word"]
#         lemma = token["lemma"]
#         dep_idx = token["index"]
            
            
#         # determine whether the sentence contains a fatal mention, whether through verbs or adjective
#         if pos in VERBS:
#             if lemma not in ALL_FATAL_VERBS:
#                 #print("VERB BUT NOT FATAL - SKIPPING.")
#                 continue
#         else:
#             if pos in ADJECTIVES:
#                 if lemma not in FATAL_ADJECTIVES:
#                     #print("ADJ BUT NOT FATAL - SKIPPING.")
#                     continue
#             else:
#                 #print("NOT VERB OR ADJ - SKIPPING.")
#                 continue


#         verb_active = False if lemma in FATAL_VERBS_PASSIVE else True

#         # Find subject
#         voice = None
#         for dep in dependencies_by_governor[dep_idx]:
#             # If the sentence is in active voice, a 'nsubj' dependecy should exist.
#             # If the sentence is in passive voice a 'nsubjpass' dependency should exist
#             if dep["dep"] == "nsubj" or dep["dep"] == "csubj":
#                 voice = "ACTIVE"
#                 break
#             elif dep["dep"] == "nsubj:pass" or dep["dep"] == "csubj:pass" or dep["dep"] == "aux:pass":
#                 voice = "PASSIVE"
#                 break

#         # for ACTIVE verbs, we are looking for dobj is used actively or nsubjpass if used passively
#         # for PASSIVE verbs, we are looking for nsubj if used actively or nsubjpass if used passively
#         subject = None
#         perp_keyword = None
#         for dep in dependencies_by_governor[dep_idx]:
#             if voice == "ACTIVE" and not verb_active and dep["dep"] == "nsubj":
#                 # Example --> She died
#                 subject = (dep["dependent"], dep["dependentGloss"])
#                 break
#             if voice == "PASSIVE" and dep["dep"] == "nsubj:pass":
#                 # Example --> She is deceased, She was killed
#                 subject = (dep["dependent"], dep["dependentGloss"])
#                 #                         perp_keyword = "iobj" NEED TO CHECK THIS
#                 break
#             if verb_active and dep["dep"] == "obj":
#                 voice = "ACTIVE"
#                 # Example --> He killed her
#                 subject = (dep["dependent"], dep["dependentGloss"])
#                 #                         perp_keyword = "nsubj"
#                 break

#         # If subject found, find GUESS for subject's affiliation
#         IS_MEMBER_PALESTINE = False
#         IS_MEMBER_ISRAEL = False
#         if subject is not None:
#             subj_idx = subject[0]
#             subj_token = tokens_by_idx[subj_idx]
#             print("SUBJECT", subject[1], subj_token["ner"])
#             subject_descriptors = investigate_subject(subject, dependencies_by_governor)
#             # Check prepositional phrases for this sentence
#             if prepositional_information is None:
#                 prepositional_information = set()
#                 for dep in dependencies:
#                     if (dep["dep"] == "case"):
#                         if (tokens_by_idx[dep["dependent"]]["lemma"] == "at"
#                                 or tokens_by_idx[dep["dependent"]]["lemma"] == "in"):
#                             prepositional_information.add(dep["governorGloss"])
#             for pi in prepositional_information:
#                 subject_descriptors[2].add(pi)

#             # Finally, check prepositional phrases in the sentence
#             # token --> case --> to preposition (in or at)
#             ### ---------------TODO: MUST CHANGE HERE WHEN DATASET UPDATES
#             for j in range(3):
#                 for sd in subject_descriptors[j]:
#                     if sd in PALESTINE_MEMBER_AFFILIATIONS:
#                         IS_MEMBER_PALESTINE = True
#                     if sd in ISRAEL_MEMBER_AFFILIATIONS:
#                         IS_MEMBER_ISRAEL = True

#         # MANUALLY VALIDATE CATEGORY
#         if sentence_text not in assigned_sentences:
#             print(index, '/', to_annotate)
#             print('Please assign a category to the VICTIM:')
#             if (IS_MEMBER_PALESTINE or IS_MEMBER_ISRAEL) and not (IS_MEMBER_PALESTINE and IS_MEMBER_ISRAEL):
#                 category_guess = 'palestine' if IS_MEMBER_PALESTINE else 'israel'
#                 print('My guess is the VICTIM is from', category_guess)
#             assigned = False
#             sentence_chunk = " ".join(
#                 text_all_sentences[max(0, sentence_index - 3):min(len(sentences) - 1, sentence_index + 4)])
#             for string in [sentence_text, sentence_chunk, article_text]:
#                 print(string)
#                 while True:
#                     mapper = {'pal':'palestine', 'is':'israel'}
#                     category = input(
#                         "Enter category ('palestine', 'israel', 'both', none', or 'next' for the next string): ").strip().lower()
#                     print()
#                     if category in mapper:
#                         category = mapper[category]
#                     print('You chose', category)
#                     if category == 'palestine' or category == 'israel' or category == 'both':
#                         categories.append(category)
#                         assigned = True
#                         break
#                     elif category == 'none':
#                         assigned = True
#                         break
#                     elif category == 'next':
#                         break
#                     else:
#                         print("Invalid category. Please enter 'palestine', 'israel', 'both', 'none', or 'next'.")
#                 if assigned:
#                     print('Categorized as', category)
#                     break

#             refresh_screen()
#             print()
#         else:
#             print('ASSIGNED TO EXISTING CATEGORY')
#             assigned = True
#             category = assigned_sentences[sentence_text]
#             categories.append(category)

#         if assigned and category != 'none':
#             # date_str = extract_date(index, file_count)
#             # date = parse_date(date_str)
#             dates.append(date)
#             voices.append(voice)
#             recorded_sentences.append(sentence_text)

#             # metadata
#             titles.append(title)
#             _ids.append(_id)

#             # add to dictionary to handle duplicates
#             assigned_sentences[sentence_text] = category
            
# # After all sentences are complete, save info for a file
# file_dict = {
#     'article_id': _ids,
#     'article_title': titles,
#     'article_date': dates,
#     'sentence': recorded_sentences,
#     'category': categories,
#     'voice': voices
# }
# df = pd.DataFrame(file_dict)
# df.to_csv(root_directory + "fatality_counts/" + 'articles_' + str(_id) + '.csv', index=False)
# print('Saved CSV file, moving on to next article...', index + 1)
# print('.......')
# refresh_screen()
# print()

• Doden bij aanvallen op kampen Centraal-Gaza




In [19]:
for doc in docs:
    print(doc.ents)

(Gaza, Rafah-grens)
(AFP, NOS, dinsdag, 7 mei, 21:18
, Gaza, Palestijnen, Gaza, twee, Israël, Rafah, Kerem Shalom, Gaza, zondag, Hamas, Gaza, VN, Noord-Gaza, UNRWA, X., OCHA, zeven maanden durende, Michiel Servaes, Oxfam, Novib, Nederland, twee miljoen Gazanen", eerste weken, 7 oktober, Gaza, Servaes, Gaza, vijf, twee, Erez, Gaza, OCHA, Gaza, één, benzine, maart, IPC, 1,1 miljoen Gazanen, 16 maart, 15 juli, 5, waarschuwingsniveau, IPC, NOS, Human Rights Watch, Oxfam Novib, Israël, NOS, 3, Gaza, Gaza, Israël, OCHA, 340, Gaza, 500, vrachtladingen, eerste, NOS, UNRWA, Gaza, Gazanen, Rafah, Israël, Oost-Rafah, Younis, Gazastrook, 23, NOS, Gazanen, Servaes, Oxfam, drie, vier dagen, Israël, Rafah, Egypte, Israël, Egypte, Gaza, Oorlog, Hamas-Israël, STER.NL)


In [ ]:
root_directory = './'

# list of ids we have already analysed
analysed_ids = [int(f.split(".")[0].split("_")[-1]) for f in os.listdir(root_directory + "fatality_counts/") if (f != '' and f not in ['.gitkeep','archive','summary'])]

# init to handle duplicates
assigned_sentences = {} 

to_annotate = df.shape[0]

for index, row in df.iterrows():

    _ids = []
    titles = []
    dates = []
    voices = []
    categories = []
    recorded_sentences = []
    
    # results_file = row['results_file']
    # article_file = row['article_file']
    _id = row['id']
    
    
    if _id in analysed_ids:
        continue

    # # Open NLP-analyzed result
    # filename = root_directory + results_file
    # try:
    #     with open(filename) as d:
    #         print(filename)
    #         data = json.load(d)
    # except FileNotFoundError:
    #     print('FILE NOT FOUND')
    #     continue

    # # Open original text block from preprocessed data file
    # original_filename = root_directory + article_file
    # f = open(original_filename, "r")
    # article_text = f.read()
    # f.close()

    # Extract original date
    title, date = row['title'], row['date'].date().strftime('%Y-%m-%d')

    # Extract NLP results
    sentences = data["sentences"]
    # doc = nlp('Twee Palestijnen doodgeschoten op Westelijke Jordaanoever')
    sentences = [sent.text for sent in doc.sents]
    text_all_sentences = extract_sentences(sentences)
    
    #for s in text_all_sentences:
    #    print(s, '\n')
    
    # Iterate through POS labels for each token
    file_count = -1
    fn_started = False
    
    for sentence in sentences:
        #print(sentence)
        sentence_index = sentence["index"]

        tokens = sentence["tokens"]
        dependencies = sentence["basicDependencies"]
        sentence_text = ""
        for token in tokens:
            sentence_text += token["before"] + token["word"] + token["after"]
            
        #print(sentence_text)

        # create a data structure that maps governor dep_idx to dependencies
        dependencies_by_governor = [[] for i in range(len(tokens) + 1)]
        for dep in dependencies:
            gov_idx = dep["governor"]
            dependencies_by_governor[gov_idx].append(dep)

        # create data structure for tokens
        tokens_by_idx = {}
        for token in tokens:
            tokens_by_idx[token["index"]] = token

        prepositional_information = None
        for token in tokens:
            pos = token["pos"]
            word = token["word"]
            lemma = token["lemma"]
            dep_idx = token["index"]
                
                
            # determine whether the sentence contains a fatal mention, whether through verbs or adjective
            if pos in VERBS:
                if lemma not in ALL_FATAL_VERBS:
                    #print("VERB BUT NOT FATAL - SKIPPING.")
                    continue
            else:
                if pos in ADJECTIVES:
                    if lemma not in FATAL_ADJECTIVES:
                        #print("ADJ BUT NOT FATAL - SKIPPING.")
                        continue
                else:
                    #print("NOT VERB OR ADJ - SKIPPING.")
                    continue


            verb_active = False if lemma in FATAL_VERBS_PASSIVE else True

            # Find subject
            voice = None
            for dep in dependencies_by_governor[dep_idx]:
                # If the sentence is in active voice, a 'nsubj' dependecy should exist.
                # If the sentence is in passive voice a 'nsubjpass' dependency should exist
                if dep["dep"] == "nsubj" or dep["dep"] == "csubj":
                    voice = "ACTIVE"
                    break
                elif dep["dep"] == "nsubj:pass" or dep["dep"] == "csubj:pass" or dep["dep"] == "aux:pass":
                    voice = "PASSIVE"
                    break

            # for ACTIVE verbs, we are looking for dobj is used actively or nsubjpass if used passively
            # for PASSIVE verbs, we are looking for nsubj if used actively or nsubjpass if used passively
            subject = None
            perp_keyword = None
            for dep in dependencies_by_governor[dep_idx]:
                if voice == "ACTIVE" and not verb_active and dep["dep"] == "nsubj":
                    # Example --> She died
                    subject = (dep["dependent"], dep["dependentGloss"])
                    break
                if voice == "PASSIVE" and dep["dep"] == "nsubj:pass":
                    # Example --> She is deceased, She was killed
                    subject = (dep["dependent"], dep["dependentGloss"])
                    #                         perp_keyword = "iobj" NEED TO CHECK THIS
                    break
                if verb_active and dep["dep"] == "obj":
                    voice = "ACTIVE"
                    # Example --> He killed her
                    subject = (dep["dependent"], dep["dependentGloss"])
                    #                         perp_keyword = "nsubj"
                    break

            # If subject found, find GUESS for subject's affiliation
            IS_MEMBER_PALESTINE = False
            IS_MEMBER_ISRAEL = False
            if subject is not None:
                subj_idx = subject[0]
                subj_token = tokens_by_idx[subj_idx]
                print("SUBJECT", subject[1], subj_token["ner"])
                subject_descriptors = investigate_subject(subject, dependencies_by_governor)
                # Check prepositional phrases for this sentence
                if prepositional_information is None:
                    prepositional_information = set()
                    for dep in dependencies:
                        if (dep["dep"] == "case"):
                            if (tokens_by_idx[dep["dependent"]]["lemma"] == "at"
                                    or tokens_by_idx[dep["dependent"]]["lemma"] == "in"):
                                prepositional_information.add(dep["governorGloss"])
                for pi in prepositional_information:
                    subject_descriptors[2].add(pi)

                # Finally, check prepositional phrases in the sentence
                # token --> case --> to preposition (in or at)
                ### ---------------TODO: MUST CHANGE HERE WHEN DATASET UPDATES
                for j in range(3):
                    for sd in subject_descriptors[j]:
                        if sd in PALESTINE_MEMBER_AFFILIATIONS:
                            IS_MEMBER_PALESTINE = True
                        if sd in ISRAEL_MEMBER_AFFILIATIONS:
                            IS_MEMBER_ISRAEL = True

            # MANUALLY VALIDATE CATEGORY
            if sentence_text not in assigned_sentences:
                print(index, '/', to_annotate)
                print('Please assign a category to the VICTIM:')
                if (IS_MEMBER_PALESTINE or IS_MEMBER_ISRAEL) and not (IS_MEMBER_PALESTINE and IS_MEMBER_ISRAEL):
                    category_guess = 'palestine' if IS_MEMBER_PALESTINE else 'israel'
                    print('My guess is the VICTIM is from', category_guess)
                assigned = False
                sentence_chunk = " ".join(
                    text_all_sentences[max(0, sentence_index - 3):min(len(sentences) - 1, sentence_index + 4)])
                for string in [sentence_text, sentence_chunk, article_text]:
                    print(string)
                    while True:
                        mapper = {'pal':'palestine', 'is':'israel'}
                        category = input(
                            "Enter category ('palestine', 'israel', 'both', none', or 'next' for the next string): ").strip().lower()
                        print()
                        if category in mapper:
                            category = mapper[category]
                        print('You chose', category)
                        if category == 'palestine' or category == 'israel' or category == 'both':
                            categories.append(category)
                            assigned = True
                            break
                        elif category == 'none':
                            assigned = True
                            break
                        elif category == 'next':
                            break
                        else:
                            print("Invalid category. Please enter 'palestine', 'israel', 'both', 'none', or 'next'.")
                    if assigned:
                        print('Categorized as', category)
                        break

                refresh_screen()
                print()
            else:
                print('ASSIGNED TO EXISTING CATEGORY')
                assigned = True
                category = assigned_sentences[sentence_text]
                categories.append(category)

            if assigned and category != 'none':
                # date_str = extract_date(index, file_count)
                # date = parse_date(date_str)
                dates.append(date)
                voices.append(voice)
                recorded_sentences.append(sentence_text)

                # metadata
                titles.append(title)
                _ids.append(_id)

                # add to dictionary to handle duplicates
                assigned_sentences[sentence_text] = category
                
    # After all sentences are complete, save info for a file
    file_dict = {
        'article_id': _ids,
        'article_title': titles,
        'article_date': dates,
        'sentence': recorded_sentences,
        'category': categories,
        'voice': voices
    }
    df = pd.DataFrame(file_dict)
    df.to_csv(root_directory + "fatality_counts/" + 'articles_' + str(_id) + '.csv', index=False)
    print('Saved CSV file, moving on to next article...', index + 1)
    print('.......')
    refresh_screen()
    print()